<a href="https://colab.research.google.com/github/reyhannaufal/big-data/blob/main/week-3/pyspark__LSH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LSH Problem
Reyhan Naufal Rahman - 0511194000171

### Import Stuff and Environment

In [53]:
import pandas as pd
from pyspark.sql import SparkSession

In [1]:
!java --version
!python --version

openjdk 11.0.18 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
Python 3.8.10


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=73a28e5ee1cbaac48a6632cdfd064572f7d86b07a161547285db88ec765dfe50
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [3]:
# Create Spark Session/Context
# We are using local machine with all the CPU cores [*]
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Hello Pyspark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
# Check spark session
print(spark)

In [5]:
!mkdir ~/.kaggle/
!touch ~/.kaggle/kaggle.json

# PLEASE USE YOUR OWN KEY
# Download your own key according to this instruction https://github.com/Kaggle/kaggle-api#api-credentials

api_token = {"username":"reyhannaufalr","key":"60656270f29453eec35ce5d9200e91f2"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d urbanbricks/wikipedia-promotional-articles

 97% 196M/201M [00:01<00:00, 118MB/s]
100% 201M/201M [00:01<00:00, 131MB/s]


In [7]:
!unzip wikipedia-promotional-articles.zip

Archive:  wikipedia-promotional-articles.zip
  inflating: good.csv                
  inflating: promotional.csv         


In [8]:
!ls -la

total 783144
drwxr-xr-x 1 root root      4096 Mar  3 13:36 .
drwxr-xr-x 1 root root      4096 Mar  3 13:30 ..
drwxr-xr-x 4 root root      4096 Mar  1 14:33 .config
-rw-r--r-- 1 root root 475685227 Oct 27  2019 good.csv
-rw-r--r-- 1 root root 115360355 Oct 27  2019 promotional.csv
drwxr-xr-x 1 root root      4096 Mar  1 14:34 sample_data
-rw-r--r-- 1 root root 210863294 Mar  3 13:35 wikipedia-promotional-articles.zip


In [9]:
# Read CSV
df = spark.read.option("header", True).csv("/content/promotional.csv")
df.printSchema()

root
 |-- text: string (nullable = true)
 |-- advert: string (nullable = true)
 |-- coi: string (nullable = true)
 |-- fanpov: string (nullable = true)
 |-- pr: string (nullable = true)
 |-- resume: string (nullable = true)
 |-- url: string (nullable = true)



In [11]:
# Add an ID for the dataset
from pyspark.sql.functions import monotonically_increasing_id

promotions_df = df.withColumn("id", monotonically_increasing_id())
promotions_df.show()

+--------------------+------+---+------+---+------+--------------------+---+
|                text|advert|coi|fanpov| pr|resume|                 url| id|
+--------------------+------+---+------+---+------+--------------------+---+
|1 Litre no Namida...|     0|  0|     1|  0|     0|https://en.wikipe...|  0|
|1DayLater was fre...|     1|  1|     0|  0|     0|https://en.wikipe...|  1|
|1E is a privately...|     1|  0|     0|  0|     0|https://en.wikipe...|  2|
|1Malaysia pronoun...|     1|  0|     0|  0|     0|https://en.wikipe...|  3|
|The Jerusalem Bie...|     1|  0|     0|  0|     0|https://en.wikipe...|  4|
|1st Round Enterpr...|     0|  0|     0|  1|     0|https://en.wikipe...|  5|
|2ergo is a provid...|     1|  0|     0|  0|     0|https://en.wikipe...|  6|
|2N Telekomunikace...|     1|  0|     0|  0|     0|https://en.wikipe...|  7|
|A 3D printing mar...|     1|  0|     0|  0|     0|https://en.wikipe...|  8|
|3DR is an America...|     1|  1|     0|  0|     0|https://en.wikipe...|  9|

In [13]:
promotions_df.count()

23837

### Tokenizer

In [15]:
# Prepare the tokenizer
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsDF = tokenizer.transform(promotions_df)

wordsDF.show()

+--------------------+------+---+------+---+------+--------------------+---+--------------------+
|                text|advert|coi|fanpov| pr|resume|                 url| id|               words|
+--------------------+------+---+------+---+------+--------------------+---+--------------------+
|1 Litre no Namida...|     0|  0|     1|  0|     0|https://en.wikipe...|  0|[1, litre, no, na...|
|1DayLater was fre...|     1|  1|     0|  0|     0|https://en.wikipe...|  1|[1daylater, was, ...|
|1E is a privately...|     1|  0|     0|  0|     0|https://en.wikipe...|  2|[1e, is, a, priva...|
|1Malaysia pronoun...|     1|  0|     0|  0|     0|https://en.wikipe...|  3|alaysia, prono...|
|The Jerusalem Bie...|     1|  0|     0|  0|     0|https://en.wikipe...|  4|[the, jerusalem, ...|
|1st Round Enterpr...|     0|  0|     0|  1|     0|https://en.wikipe...|  5|[1st, round, ente...|
|2ergo is a provid...|     1|  0|     0|  0|     0|https://en.wikipe...|  6|[2ergo, is, a, pr...|
|2N Telekomunikace...| 

In [27]:
# Vectorize the dataset
from pyspark.ml.feature import CountVectorizer

vocabSize=2000

# Train the CountVectorizer Model using our data
cvModel = CountVectorizer(inputCol="words", outputCol="features", vocabSize=vocabSize, minDF=5).fit(wordsDF)

# Transform our data into vector
vectorizedDF = cvModel.transform(wordsDF)
vectorizedDF.show()

+--------------------+------+---+------+---+------+--------------------+---+--------------------+--------------------+
|                text|advert|coi|fanpov| pr|resume|                 url| id|               words|            features|
+--------------------+------+---+------+---+------+--------------------+---+--------------------+--------------------+
|1 Litre no Namida...|     0|  0|     1|  0|     0|https://en.wikipe...|  0|[1, litre, no, na...|(2000,[0,1,2,3,4,...|
|1DayLater was fre...|     1|  1|     0|  0|     0|https://en.wikipe...|  1|[1daylater, was, ...|(2000,[0,1,2,3,4,...|
|1E is a privately...|     1|  0|     0|  0|     0|https://en.wikipe...|  2|[1e, is, a, priva...|(2000,[0,1,2,3,4,...|
|1Malaysia pronoun...|     1|  0|     0|  0|     0|https://en.wikipe...|  3|alaysia, prono...|(2000,[0,1,2,3,4,...|
|The Jerusalem Bie...|     1|  0|     0|  0|     0|https://en.wikipe...|  4|[the, jerusalem, ...|(2000,[0,1,2,3,4,...|
|1st Round Enterpr...|     0|  0|     0|  1|     0|

### Fit/Train Model LSH

In [28]:
from  pyspark.ml.feature import MinHashLSH

mh = MinHashLSH(inputCol="features", outputCol="hashValues", numHashTables=3)
LSHmodel = mh.fit(vectorizedDF)

LSHmodel.transform(vectorizedDF).show()

+--------------------+------+---+------+---+------+--------------------+---+--------------------+--------------------+--------------------+
|                text|advert|coi|fanpov| pr|resume|                 url| id|               words|            features|          hashValues|
+--------------------+------+---+------+---+------+--------------------+---+--------------------+--------------------+--------------------+
|1 Litre no Namida...|     0|  0|     1|  0|     0|https://en.wikipe...|  0|[1, litre, no, na...|(2000,[0,1,2,3,4,...|[[2.8153831E7], [...|
|1DayLater was fre...|     1|  1|     0|  0|     0|https://en.wikipe...|  1|[1daylater, was, ...|(2000,[0,1,2,3,4,...|[[1958739.0], [83...|
|1E is a privately...|     1|  0|     0|  0|     0|https://en.wikipe...|  2|[1e, is, a, priva...|(2000,[0,1,2,3,4,...|[[1.0250731E7], [...|
|1Malaysia pronoun...|     1|  0|     0|  0|     0|https://en.wikipe...|  3|alaysia, prono...|(2000,[0,1,2,3,4,...|[[2.8153831E7], [...|
|The Jerusalem Bie...| 

In [48]:
print(cvModel.vocabulary.index("system"))
print(cvModel.vocabulary.index("computer"))

131
439


In [49]:
# Testing searching for "united" "states"

from pyspark.ml.linalg import Vectors


# Convert the input with 3 words into 1000 size vectors
# If the words exist in the index we will give value = 1.0, otherwise 0.0
# Final result: key = [0, 0, ... , 1.0, ..., 1.0, 1.0, ....]

key = Vectors.sparse(vocabSize, {cvModel.vocabulary.index("system"): 1.0, cvModel.vocabulary.index("computer"): 1.0})

In [50]:
# Define the number of neighbours
k = 40

# Search inside LSH model that we already trained
resultDF = LSHmodel.approxNearestNeighbors(vectorizedDF, key, k)
resultDF.show()

+--------------------+------+---+------+---+------+--------------------+----------+--------------------+--------------------+--------------------+------------------+
|                text|advert|coi|fanpov| pr|resume|                 url|        id|               words|            features|          hashValues|           distCol|
+--------------------+------+---+------+---+------+--------------------+----------+--------------------+--------------------+--------------------+------------------+
|Newrite is a syst...|     1|  0|     0|  0|     0|https://en.wikipe...|8589937035|[newrite, is, a, ...|(2000,[0,1,2,3,4,...|[[3.9084055E7], [...|0.9555555555555556|
|Continuous Variab...|     1|  0|     0|  0|     0|https://en.wikipe...|      4902|[continuous, vari...|(2000,[0,3,4,5,6,...|[[5.6327597E7], [...|0.9565217391304348|
|TrueBeam is a rad...|     1|  0|     0|  0|     0|https://en.wikipe...|8589943831|[truebeam, is, a,...|(2000,[0,1,2,5,6,...|[[1.61060448E8], ...|0.9655172413793104|
|The

In [51]:
data = resultDF.toPandas()
data.to_csv("result.csv")

In [52]:
df = pd.read_csv('result.csv')
df

,Unnamed: 0,text,advert,coi,fanpov,pr,resume,url,id,words,features,hashValues,distCol
0,0,Newrite is a system of shorthand invented by t...,1,0,0,0,0,https://en.wikipedia.org/wiki/Newrite,8589937035,"['newrite', 'is', 'a', 'system', 'of', 'shorth...","(2000,[0,1,2,3,4,5,6,8,9,10,11,12,16,18,19,20,...","[DenseVector([39084055.0]), DenseVector([21571...",0.955556
1,1,Continuous Variable Valve Timing Control Syste...,1,0,0,0,0,https://en.wikipedia.org/wiki/CVVTCS,4902,"['continuous', 'variable', 'valve', 'timing', ...","(2000,[0,3,4,5,6,9,11,12,16,19,23,57,87,123,13...","[DenseVector([56327597.0]), DenseVector([21571...",0.956522
2,2,"TrueBeam is a radiotherapy device, a linear ac...",1,0,0,0,0,https://en.wikipedia.org/wiki/Truebeam,8589943831,"['truebeam', 'is', 'a', 'radiotherapy', 'devic...","(2000,[0,1,2,5,6,7,10,12,19,24,25,31,32,38,53,...","[DenseVector([161060448.0]), DenseVector([2157...",0.965517
3,3,The Vybrid Series is a low power System on chi...,1,0,0,0,0,https://en.wikipedia.org/wiki/Vybrid%20Series,8589944900,"['the', 'vybrid', 'series', 'is', 'a', 'low', ...","(2000,[0,1,4,5,6,9,10,11,14,15,57,85,97,126,12...","[DenseVector([56327597.0]), DenseVector([21571...",0.965517
4,4,FUNTORO is a Taiwan based subsidiary brand of ...,1,0,0,0,0,https://en.wikipedia.org/wiki/FUNTORO,7426,"['funtoro', 'is', 'a', 'taiwan', 'based', 'sub...","(2000,[0,1,2,4,5,6,7,11,16,19,23,40,54,55,59,8...","[DenseVector([9591173.0]), DenseVector([215711...",0.972973
5,5,Periowave is an antimicrobial photodynamic the...,1,0,0,0,0,https://en.wikipedia.org/wiki/Periowave,8589938411,"['periowave', 'is', 'an', 'antimicrobial', 'ph...","(2000,[0,1,2,3,4,5,6,7,8,9,10,12,13,16,19,20,2...","[DenseVector([38094718.0]), DenseVector([21571...",0.978723
6,6,Recoll is a desktop search tool that provides ...,1,0,0,0,0,https://en.wikipedia.org/wiki/Recoll,8589939589,"['recoll', 'is', 'a', 'desktop', 'search', 'to...","(2000,[0,1,2,3,4,5,6,7,8,9,10,11,13,14,15,19,2...","[DenseVector([56327597.0]), DenseVector([21571...",0.980769
7,7,Actibump is a traffic safety system where spee...,1,0,0,0,0,https://en.wikipedia.org/wiki/Actibump,198,"['actibump', 'is', 'a', 'traffic', 'safety', '...","(2000,[0,1,3,4,5,6,7,11,12,13,16,18,19,20,23,2...","[DenseVector([969402.0]), DenseVector([2157114...",0.983871
